In [27]:
# https://news.cnyes.com/news/cat/tw_stock_news

In [28]:
# 操作 browser 的 驅動程式
from selenium import webdriver

# 負責開啟和關閉 Chrome 的套件
from selenium.webdriver.chrome.service import Service

# 自動下載 Chrome Driver 的套件
from webdriver_manager.chrome import ChromeDriverManager

# 啟動瀏覽器工具的選項
from selenium.webdriver.chrome.options import Options

# 例外處理的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待、了解某個元素的狀態，通常與 exptected_conditions 和 By 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，經常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 隨機取得 User-Agent
from fake_useragent import UserAgent
# ua = UserAgent()

import pymysql
from pymysql.err import IntegrityError
from time import sleep
import json
import random
import os
from datetime import datetime
from datetime import timedelta
import re
from pprint import pprint


In [29]:
user_agent_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393",
    "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 6P Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36"
    "Mozilla/5.0 (iPhone14,3; U; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) Version/10.0 Mobile/19A346 Safari/602.1"
    # Add more user agent strings as needed
]
user_agent = random.choice(user_agent_list)

my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")  # 最大化視窗
my_options.add_argument("--incognito")  # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")  # 取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  # 設定為正體中文
my_options.add_argument('blink-settings=imagesEnabled=false')  # 不載入圖
my_options.add_argument(f'--user-agent={user_agent}')  # 隨機 user-angent 
my_options.add_experimental_option(
    "excludeSwitches", ['enable-automation', 'enable-logging'])  # 沒有異常log


driver = webdriver.Chrome(
    options=my_options,
    service=Service(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|██████████| 6.78M/6.78M [00:02<00:00, 3.39MB/s]


In [30]:
# 建立下載路徑/資料夾，不存在就新增 (os.getcwd() 會取得當前的程式工作目錄)
folderPath = os.path.join(os.getcwd(), 'crawler_data')
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

#預設下載路徑
my_options.add_experimental_option("prefs", {
  "download.default_directory": folderPath
})

In [31]:

url = 'https://news.cnyes.com/news/cat/tw_stock_news'

listData = []
results = []
alinks = []

In [32]:
def visit():
    driver.get(url)


def scroll():

    innerHeight = 0
    offset = 0
    count = 0
    limit = 3

    while count <= limit:
        offset = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )

        # 捲軸往下滑動
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        ''')

        # 強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
        sleep(3)

        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )

        # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset == innerHeight:
            count += 1

        # 為了實驗功能，捲動超過一定的距離，就結束程式
        if offset >= 500:
            break

def inputlink():
    links = driver.find_elements(
        By.CSS_SELECTOR, 
        "div > a._1Zdp"
    )
    times = driver.find_elements(
        By.CSS_SELECTOR, 
        "div > a._1Zdp > div > time"
    )
    
    ctime = datetime.now()
    today0am = ctime.replace(hour=0, minute=0, second=0, microsecond=0)
    yesterday0am = today0am - timedelta(days=1)


    print("before",times[-1].text)
    # print(type(times[-1].text))
    try:
        while ":" in times[-1].text:
            scroll()

            sleep(4)
            
            times = driver.find_elements(
            By.CSS_SELECTOR, 
            "div > a._1Zdp > div > time"
            )

            links = driver.find_elements(
            By.CSS_SELECTOR, 
            "div > a._1Zdp"
            )

            check_links = driver.find_elements(
            By.CSS_SELECTOR,
            "div._2bFl.theme-list > div"
            )

            print("after",times[-1].text)
        # sleep(2)
        
        # print(times[-1].text)
        # print(type(times[-1].text))
    except:
        pass
        

    
    for i in range(len(times)):
        article_datetime = times[i].get_attribute('innerText')
        # print(article_datetime)

        if ":" in article_datetime:
            time_hour = article_datetime.split(":")[0]
            time_mins = article_datetime.split(":")[1]
            post_time = ctime.replace(hour=int(time_hour), minute=int(time_mins), second=0, microsecond=0)

            href = links[i].get_attribute('href')
            alinks.append(href)
            
            print(post_time)
        else: 
            pass
        

In [33]:
def craw(link):

    # title
    title = driver.find_element(
        By.CSS_SELECTOR, '#content > div > div > div._2hZZ.theme-app.theme-newsdetail > main > div._uo1n._2l9x > h1').get_attribute('innerText')
    # time
    article_datetime = driver.find_element(
        By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[2]/div[2]/time').get_attribute('datetime')

    article_date_str = re.split('\+', article_datetime)[0].replace('T', ' ')

    # article
    article_list = driver.find_elements(
        By.CSS_SELECTOR, 'div > div._2E8y > p')
 
    article = ''
    for a in article_list:
        article += a.get_attribute("innerText")

    # reporter
    reporter = driver.find_element(
        By.CSS_SELECTOR, "span._3lKe > span").get_attribute("innerText").split(" ")[0][5:]

    result = {
        'news': '鉅亨網',
        'link': link, 
        'title': title, 
        'time': article_date_str, 
        'reporter': reporter, 
        'article': article
        }
    
    results.append(result)

In [34]:
if __name__ == "__main__":
    visit()
    inputlink()
    for link in alinks:
        driver.get(link)
        craw(link)
        
        sleep(2)

    driver.quit() 
        

before 02/22
2023-02-23 11:53:00
2023-02-23 11:35:00
2023-02-23 11:02:00
2023-02-23 11:02:00
2023-02-23 10:39:00
2023-02-23 10:35:00
2023-02-23 10:29:00
2023-02-23 09:41:00
2023-02-23 05:10:00


In [35]:
alinks
results

[{'news': '鉅亨網',
  'link': 'https://news.cnyes.com/news/id/5094866?exp=a',
  'title': '〈焦點股〉RFID廠韋僑新廠產能即將開出 股價爆量漲停創天價',
  'time': '2023-02-23 11:53:22',
  'reporter': '張欽發',
  'article': '韋僑 (6417-TW) 在台中的建廠新投資即將開出新產能，同時，也預估 2023 年在醫療應用的利基市場的營收比重將突破 30%，加上 RFID 產業的市場能見度大增，韋僑股價在今天盤中放量以漲停 110 元創新天價。韋僑在台中大里仁化工業區投資超過 13 億元設立新廠已完工取得使用執照，第一季逐步完成各項設備進駐量產，預估 2023 年全年營收將在醫療應用領域出貨比重提高挹注下，全年營收年成長幅度還要攀升到 20% 以上。韋僑預估 2023 年第一季平均單月營收在 1 億元之上，首季有 3 億元以上的營收，年增 14%，同時，預估全年營收還將逐季攀高。韋僑在 2022 前三季營收達 9.46 億元其中醫療應用領域出貨比重成長至 29%，2023 年可望在歐系客戶擴大採購的利基推升下，比重突破 30%。同時，韋僑今年也進一步取得國際低溫物流新訂單，將成為挹注成長的動能之一。全球消費性電子市場在年下半年銷售顯露疲態，包括 NB、PC 等產品至今仍處於消化庫存階段，但無線射頻標籤在智能化、自動化管理應用持續成長，且應用領域持續擴大，有助商機成長。韋僑今年可完成遷入自建新廠集中生產後，將擴充應用鎖定票證、物流、醫療及工業製程等領域，並導入自動化生產，設計產能可達 30 億元，新廠完工，將刺激營收、獲利再成長 20% 創新高，韋僑 2022 年前三季營收達 9.46 億元，稅後純益 1.33 億元，年增 72.21%，已超越 2021 年全年，每股純益達 3.15 元。'},
 {'news': '鉅亨網',
  'link': 'https://news.cnyes.com/news/id/5094833?exp=a',
  'title': '雷虎澄清早已排除中國供應鏈 國機國造衝軍工版圖',
  'time': '2023-02-23 11:35:56',
  'reporter': '王莞甯',